<h1>Qayum Khan @ FINM36700 UChicago, Autumn 2022<br>
Homework 7 : Estimating the Equity Risk Premium</h1>

In [1]:
import pandas as pd
pd.set_option("display.precision", 4)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from math import sqrt
import numpy as np
import seaborn as sbn

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

In [2]:
# SOURCE : class GitHub cmds folder
def performance_summary(facurn_data):
    """ 
        facurns the Performance Stats for given set of facurns
        Inputs: 
            facurn_data - DataFrame with Date index and Monthly facurns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean facurn, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly facurns. 
    """
    summary_stats = facurn_data.mean().to_frame('Annualized Return').apply(lambda x: x*12)
    summary_stats['Annualized Volatility'] = facurn_data.std().apply(lambda x: x*np.sqrt(12))
    summary_stats['Annualized Sharpe Ratio'] = summary_stats['Annualized Return']/summary_stats['Annualized Volatility']
    
    summary_stats['Skewness'] = facurn_data.skew()
    summary_stats['Excess Kurtosis'] = facurn_data.kurtosis()
    summary_stats['VaR (0.05)'] = facurn_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = facurn_data[facurn_data <= facurn_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = facurn_data.min()
    summary_stats['Max'] = facurn_data.max()
    
    wealth_index = 1000*(1+facurn_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [3]:
ret_sig = pd.read_excel( 'gmo_analysis_data.xlsx', 1, index_col=0 ).dropna()
ret_tot = pd.read_excel( 'gmo_analysis_data.xlsx', 2, index_col=0 ).dropna()
ret_RF = pd.read_excel( 'gmo_analysis_data.xlsx', 3, index_col=0 ).dropna()

<h1>SECTION 1 (optional): GMO &mdash; Grantham, Mayo & Van Otterloo</h1>

<h2>QUESTION 1 : GMO's Approach</h2>

(a)  GMO believes that they can more easily predict long-run than short-run asset-class performance, because the market in the long-run is like a <i>weighing machine</i> (convergence toward <b>fundamental value</b> for value-driven investors), whereas the market in the short-run is like a <i>voting machine</i> (noisy).

(b)  GMO's full model uses the following predicting variables : <b>D/P ratio</b> ("dividend yield"), <b>P/E growth</b>, <b>E/S growth</b> ("profit margins"), and <b>sales growth</b> (per share).  Yes, this fits with the goal of long-run forecasts, since it is consisent with the Gordon growth model.

(c) This is a "Zen" approach (towards a steady state) against ephemeral overreaction, which ends up being contrary to human-sentiment decision-making.

(d)  This approach raises business risk and managerial-career risk, because investors and managers may tend to overreact so sell their shares of GMWAX if contrary to perceived personal gain.

<h2>QUESTION 2 : The Market Environment</h2>

(a) The so-called <b>Lost Decade (2000&ndash;2011)</b> where stocks underperformed riskfree bonds is a reason to be skeptical that the market risk premium will be as high as the past 50 years.

(b)  The biggest drivers of GMO's 2007 forecast of real excess equity returns being negative are that the signals of the <b>P/E & E/S ratios</b> were negative for the next 7 years, whereas for the steady state they are taken as zero.

(c)  In GMO's 2011 forecast, the signal that GMO has most revised relative to 2007 is the <b>P/E ratio</b> for the next 7 years.  Now, the equity premium is forecast to be <b>positive on both</b> time horizons (0 < 7-year @ 1.6% < steady-state @ 4.2%).

<h2>QUESTION 3 : Exhibit 1 (2002&ndash;2011)</h2>

(a)  <b>US equities</b> were estimated to have a negative 10-yr return. 

(b) <b>Foreign govt bonds</b> substantially outperformed GMO's estimate.

(c) <b>US Treasury bills</b> substantially underperformed GMO's estimate.

<h2>QUESTION 4 : Fund Performance (1997&ndash;2011)</h2>

(a)  <b>US fixed income</b> is the asset class GMWAX was most heavily allocated, per Exhibit 3.

(b)  GMWAX performed <b>comparably</b> to its benchmark except roughly before & after the Tech Bubble, namely 1998 and 2000&ndash;2002, per Exhibit 2.

<h1>SECTION 2 : Analyzing GMO</h1>

<h2>QUESTION 1 : Performance</h2>

In [4]:
perfA_exc = performance_summary( pd.DataFrame( ret_tot.loc[:'2011','GMWAX']-ret_RF.loc[:'2011','US3M'] ) )
perfB_exc = performance_summary( pd.DataFrame( ret_tot.loc['2012':,'GMWAX']-ret_RF.loc['2012':,'US3M'] ) )
perfC_exc = performance_summary( pd.DataFrame( ret_tot.loc[:,'GMWAX']-ret_RF.loc[:,'US3M'] ) )

perfA_tot = performance_summary( pd.DataFrame( ret_tot.loc[:'2011','GMWAX'] ) )
perfB_tot = performance_summary( pd.DataFrame( ret_tot.loc['2012':,'GMWAX'] ) )
perfC_tot = performance_summary( pd.DataFrame( ret_tot.loc[:,'GMWAX'] ) )


In [5]:
result = pd.concat( [perfA_exc, perfB_exc, perfC_exc] ).iloc[:,0:3]
result.index = ['1996--2011', '2012--2022', '1996--2022']
result.index.name = 'GMWAX excess returns'
result

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
GMWAX excess returns,,,
1996--2011,0.0158,0.1250,0.1266
2012--2022,0.0366,0.0920,0.3982
1996--2022,0.0245,0.1123,0.2181


YES, since inception, the risk premium has increased significantly after 2011 as well as the volatility decreasing, so the Sharpe ratio has more than tripled.

<h2>QUESTION 2 : Tail Risk</h2>

In [6]:
result1 = pd.concat( [perfA_exc, perfB_exc, perfC_exc] ).iloc[:,[7,5]]
result1.index = result.index
result2 = pd.concat( [perfA_tot, perfB_tot, perfC_tot] ).iloc[:,[9]]
result2.index = result.index
pd.concat( [result1, result2], axis=1 )

,Min,VaR (0.05),Max Drawdown
GMWAX excess returns,,,
1996--2011,-0.1492,-0.0598,-0.3552
2012--2022,-0.1187,-0.0397,-0.2168
1996--2022,-0.1492,-0.0483,-0.3552


(a) GMWAX has relatively <b>low tail risk</b> with MDD > -1% though -VaR > risk premia above.

(b)  <b>YES</b>, the tail risk is noticeably reduced across all three metrics after 2011.

<h2>QUESTION 3 : CAPM Regression</h2>

In [7]:
def CAPM_stats( ret_exc, mkt_exc ) :
    model = sm.OLS( ret_exc, sm.add_constant( mkt_exc ), missing='drop' ).fit()
    return pd.DataFrame( index=ret_exc.columns, columns=[r'$\alpha_\mathrm{MKT}$', r'$\beta_\mathrm{MKT}$', r'$R^2$', 'Annualized Info Ratio'], \
        data=np.array([ *model.params, model.rsquared, model.params[0]/model.resid.std() * sqrt(12) ]).reshape(1,4) )

In [8]:
result_CAPM = [None] * 3
result_CAPM[0] = CAPM_stats( (ret_tot.loc[:'2011','GMWAX']-ret_RF.loc[:'2011','US3M']).to_frame('1996--2011'), ret_tot.loc[:'2011','SPY']-ret_RF.loc[:'2011','US3M'] )
result_CAPM[1] = CAPM_stats( (ret_tot.loc['2012':,'GMWAX']-ret_RF.loc['2012':,'US3M']).to_frame('2012--2022'), ret_tot.loc['2012':,'SPY']-ret_RF.loc['2012':,'US3M'] )
result_CAPM[2] = CAPM_stats( (ret_tot.loc[:,'GMWAX']-ret_RF.loc[:,'US3M']).to_frame('1996--2022'), ret_tot.loc[:,'SPY']-ret_RF.loc[:,'US3M'] )

result_CAPM = pd.concat( result_CAPM ).iloc[:,:3]
result_CAPM.columns.name = 'GMWAX excess returns'
result_CAPM

GMWAX excess returns,$\alpha_\mathrm{MKT}$,$\beta_\mathrm{MKT}$,$R^2$
1996--2011,-0.0005,0.5396,0.5071
2012--2022,-0.0029,0.5622,0.7645
1996--2022,-0.0014,0.5456,0.5777


(b) <b>YES</b>, GMWAX is a low-$\beta$ strategy since its value is less than one for the market (SPY).  Since the 2011 case, it has only slightly increased.

(c) <b>NO</b>, GMWAX doesn't provide any positive $\alpha$ in the above monthly regression, so it's harder to justify management fees.  Since the 2011 case, it has gotten a bit worse though still hovering below zero, though the $R^2$ has significantly increased to the CAPM model.

<h1>SECTION 3 : Forecast Regressions</h1>

<h2>QUESTION 1 : Forecasting of SPY</h2>

In [9]:
model_fcA = sm.OLS( ret_tot['SPY'], sm.add_constant( ret_sig['DP'].shift(1)[ret_tot['SPY'].index[0]:] ), missing='drop' ).fit()
pd.DataFrame( index=['SPY total returns'], columns=[r'$\alpha^\mathrm{SPY}$', r'$\beta^\mathrm{SPY}_\mathrm{DP}$', r'$R^2$'], \
    data=np.array([*model_fcA.params, model_fcA.rsquared]).reshape(1,3) )

,$\alpha^\mathrm{SPY}$,$\beta^\mathrm{SPY}_\mathrm{DP}$,$R^2$
SPY total returns,-0.0142,0.0122,0.0104


In [10]:
model_fcB = sm.OLS( ret_tot['SPY'], sm.add_constant( ret_sig['EP'].shift(1)[ret_tot['SPY'].index[0]:] ), missing='drop' ).fit()
pd.DataFrame( index=['SPY total returns'], columns=[r'$\alpha^\mathrm{SPY}$', r'$\beta^\mathrm{SPY}_\mathrm{EP}$', r'$R^2$'], \
    data=np.array([*model_fcB.params, model_fcB.rsquared]).reshape(1,3) )

,$\alpha^\mathrm{SPY}$,$\beta^\mathrm{SPY}_\mathrm{EP}$,$R^2$
SPY total returns,-0.0039,0.0027,0.0056


In [11]:
model_fcC = sm.OLS( ret_tot['SPY'], sm.add_constant( ret_sig.shift(1)[ret_tot['SPY'].index[0]:] ), missing='drop' ).fit()
pd.DataFrame( index=['SPY total returns'], columns=[r'$\alpha^\mathrm{SPY}$', r'$\beta^\mathrm{SPY}_\mathrm{DP}$', r'$\beta^\mathrm{SPY}_\mathrm{EP}$', r'$\beta^\mathrm{SPY}_\mathrm{US10Y}$', r'$R^2$'], \
    data=np.array([*model_fcC.params, model_fcC.rsquared]).reshape(1,5) )

,$\alpha^\mathrm{SPY}$,$\beta^\mathrm{SPY}_\mathrm{DP}$,$\beta^\mathrm{SPY}_\mathrm{EP}$,$\beta^\mathrm{SPY}_\mathrm{US10Y}$,$R^2$
SPY total returns,-0.0183,0.0103,0.0023,-0.0007,0.0152


<h2>QUESTION 2 : Testing a Trading Strategy</h2>

In [12]:
ret_SPY_exc = (ret_tot['SPY']-ret_RF['US3M']).dropna()

ret_xA_exc = ( (100 * model_fcA.fittedvalues) * ret_SPY_exc ).to_frame( 'DP' ).dropna()
ret_xA_tot = ( (100 * model_fcA.fittedvalues) * ret_tot['SPY'] ).to_frame( 'DP' )

ret_xB_exc = ( (100 * model_fcB.fittedvalues) * ret_SPY_exc ).to_frame( 'EP' ).dropna()
ret_xB_tot = ( (100 * model_fcB.fittedvalues) * ret_tot['SPY'] ).to_frame( 'EP' )

ret_xC_exc = ( (100 * model_fcC.fittedvalues) * ret_SPY_exc ).to_frame( 'DP+EP+US10Y' ).dropna()
ret_xC_tot = ( (100 * model_fcC.fittedvalues) * ret_tot['SPY'] ).to_frame( 'DP+EP+US10Y' )

ret_x_exc = pd.concat( [ ret_xA_exc, ret_xB_exc, ret_xC_exc ], axis=1 )


In [13]:
perf_xA = pd.concat( [ performance_summary( ret_xA_exc ).iloc[:,0:3], performance_summary( ret_xA_tot ).iloc[:,[9]], CAPM_stats( ret_xA_exc,  ret_SPY_exc ) ], axis=1 )
perf_xB = pd.concat( [ performance_summary( ret_xB_exc ).iloc[:,0:3], performance_summary( ret_xB_tot ).iloc[:,[9]], CAPM_stats( ret_xB_exc,  ret_SPY_exc ) ], axis=1 )
perf_xC = pd.concat( [ performance_summary( ret_xC_exc ).iloc[:,0:3], performance_summary( ret_xC_tot ).iloc[:,[9]], CAPM_stats( ret_xC_exc,  ret_SPY_exc ) ], axis=1 )

perf_x = pd.concat( [perf_xA, perf_xB, perf_xC] )
perf_x.columns.name = "SPY strategy's excess returns"
perf_x.index.name = "forecast's signals"
perf_x

SPY strategy's excess returns,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Max Drawdown,$\alpha_\mathrm{MKT}$,$\beta_\mathrm{MKT}$,$R^2$,Annualized Info Ratio
forecast's signals,,,,,,,,
DP,0.0906,0.1690,0.5361,-0.7128,0.0020,0.8756,0.6571,106.1628
EP,0.0747,0.1248,0.5981,-0.3540,0.0018,0.7052,0.7811,144.8862
DP+EP+US10Y,0.1037,0.1567,0.6616,-0.5978,0.0037,0.7845,0.6138,96.6784


<h2>QUESTION 3 : GMO's Perception of Risk Premium</h2>

In [14]:
ret_tot.quantile(0.05, axis=0).to_frame('VaR (0.05)')

,VaR (0.05)
SPY,-0.0790
GMWAX,-0.0473


In [15]:
performance_summary( ret_x_exc['2000':'2011'] ).iloc[:,0:3]

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
DP,0.0354,0.2118,0.1669
EP,0.0145,0.1261,0.1147
DP+EP+US10Y,0.0516,0.1756,0.2937


(b) <b>YES</b>, the three dynamic portfolios from Question 2 above in Section 3 outperform the riskfree rate during the Lost Decade.

<h1>SECTION 4 : Out-of-Sample Forecasting</h1>

In [16]:
model_fcA_OOS60 = RollingOLS( ret_tot['SPY'], sm.add_constant( ret_sig['DP'].shift(1)[ret_tot['SPY'].index[0]:] ), missing='drop', expanding=True, min_nobs=60 ).fit()
model_fcB_OOS60 = RollingOLS( ret_tot['SPY'], sm.add_constant( ret_sig['EP'].shift(1)[ret_tot['SPY'].index[0]:] ), missing='drop', expanding=True, min_nobs=60 ).fit()

<h2>QUESTION 1 : Coefficient of Determination</h2>

In [17]:
print( f"The DP signal's OOS R^2 until time T=now is {model_fcA_OOS60.rsquared[-1]:.4f}.  YES, this forecasting strategy produced a positive OOS R^2." )
print( f"The EP signal's OOS R^2 until time T=now is {model_fcB_OOS60.rsquared[-1]:.4f}.  YES, this forecasting strategy produced a positive OOS R^2." )

The DP signal's OOS R^2 until time T=now is 0.0104.  YES, this forecasting strategy produced a positive OOS R^2.
The EP signal's OOS R^2 until time T=now is 0.0056.  YES, this forecasting strategy produced a positive OOS R^2.


<h2>QUESTION 2 : Testing a Trading Strategy</h2>

In [18]:
# Sadly, unlike sm.OLS's results, the results of RollingOLS don't have a "fittedvalues" property.  So must do it manually.

paramsA_OOS60 = model_fcA_OOS60.params.dropna()
signal_DP = sm.add_constant( ret_sig['DP'] ).loc[paramsA_OOS60.index[0]:]
model_fcA_OOS60_fittedvalues = pd.Series( index=paramsA_OOS60.index, data=[ paramsA_OOS60.loc[t,] @ signal_DP.loc[t,] for t in paramsA_OOS60.index ] )

paramsB_OOS60 = model_fcB_OOS60.params.dropna()
signal_EP = sm.add_constant( ret_sig['EP'] ).loc[paramsB_OOS60.index[0]:]
model_fcB_OOS60_fittedvalues = pd.Series( index=paramsB_OOS60.index, data=[ paramsB_OOS60.loc[t,] @ signal_EP.loc[t,] for t in paramsB_OOS60.index ] )


In [19]:
ret_xA_OOS60_exc = ( (100 * model_fcA_OOS60_fittedvalues) * ret_SPY_exc ).to_frame( 'DP OOS60' ).dropna()
ret_xA_OOS60_tot = ( (100 * model_fcA_OOS60_fittedvalues) * ret_tot['SPY'] ).to_frame( 'DP OOS60' )

ret_xB_OOS60_exc = ( (100 * model_fcB_OOS60_fittedvalues) * ret_SPY_exc ).to_frame( 'EP OOS60' ).dropna()
ret_xB_OOS60_tot = ( (100 * model_fcB_OOS60_fittedvalues) * ret_tot['SPY'] ).to_frame( 'EP OOS60' )

ret_x_OOS60_exc = pd.concat( [ ret_xA_OOS60_exc, ret_xB_OOS60_exc ], axis=1 )

In [20]:
perf_xA_OOS60 = pd.concat( [ performance_summary( ret_xA_OOS60_exc ).iloc[:,0:3], performance_summary( ret_xA_OOS60_tot ).iloc[:,[9]], \
    CAPM_stats( ret_xA_OOS60_exc,  ret_SPY_exc.loc[paramsA_OOS60.index[0]:] ) ], axis=1 )
perf_xB_OOS60 = pd.concat( [ performance_summary( ret_xB_OOS60_exc ).iloc[:,0:3], performance_summary( ret_xB_OOS60_tot ).iloc[:,[9]], \
    CAPM_stats( ret_xB_OOS60_exc,  ret_SPY_exc.loc[paramsB_OOS60.index[0]:] ) ], axis=1 )

perf_x_OOS60 = pd.concat( [perf_xA_OOS60, perf_xB_OOS60] )
perf_x_OOS60.columns.name = "SPY strategy's excess returns"
perf_x_OOS60.index.name = "forecast's signals"
perf_x_OOS60

SPY strategy's excess returns,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Max Drawdown,$\alpha_\mathrm{MKT}$,$\beta_\mathrm{MKT}$,$R^2$,Annualized Info Ratio
forecast's signals,,,,,,,,
DP OOS60,0.0739,0.1950,0.3793,-0.5651,-3.3924e-05,0.9267,0.515,81.9045
EP OOS60,0.1028,0.1726,0.5955,-0.4374,5.5186e-03,0.4558,0.159,34.5520


The OOS (out-of-sample) strategy is performing <b>-1.67% (DP) | +2.81% (EP) differently on annualized returns</b> compared to the IS (in-sample) versions.

<h2>QUESTION 3 : GMO's Perception of Risk Premium</h2>

In [21]:
performance_summary( ret_x_OOS60_exc['2000':'2011'] ).iloc[:,0:3]

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
DP OOS60,0.0413,0.2518,0.1641
EP OOS60,0.1162,0.2152,0.5400


(b) <b>YES</b>, the these two dynamic portfolios from Question 2 above in Section 4 outperform (DS : 0.59%, ES : +10.17%) the riskfree rate during the Lost Decade.